In [24]:
import torch
import numpy as np

In [25]:
# !pip install open-tamil
import tamil
import codecs
from tamil import utf8

## Statistical modelling

- To parse the tamil characters, we use tamil.utf8.get_letters
- this package outputs as list of tamil characters, then we can join to form the word

In [16]:
with open('./tamil_names.txt') as f:
    names_data = list(map(tamil.utf8.get_letters, list(map(str.strip, f.readlines()))))
names_data[0]

['அ', 'க', 'த்', 'தி', 'ய', 'ன்']

In [19]:
# create i_to_s and s_to_i mapping dict 
chars = ['.'] + sorted(set([j for i in names_data for j in i]))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for i,c in enumerate(chars)}
len(itos)

209

In [192]:
names_data[0]

['அ', 'க', 'த்', 'தி', 'ய', 'ன்']

In [ ]:
N = {}

In [39]:
# lets count, how many bi-gram (letter followed by another letter) occurs
for w in names_data:
    chs = ['.'] + w + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        #print(f"{ch1} {ch2}")
        ix1, ix2 = stoi[ch1], stoi[ch2]
        N[ix1, ix2] += 1

In [40]:
N

tensor([[  0,   0, 122,  ...,   2,  21,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        ...,
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0]], dtype=torch.int32)

In [49]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000, 0.0000, 0.0419, 0.0086, 0.0659, 0.0051, 0.0247, 0.0141, 0.0024,
        0.0017, 0.0069, 0.0048, 0.0758, 0.0261, 0.0103, 0.0000, 0.0268, 0.0058,
        0.0003, 0.0000, 0.0041, 0.0058, 0.0000, 0.0000, 0.0151, 0.0003, 0.0000,
        0.0000, 0.0477, 0.0058, 0.0299, 0.0000, 0.0268, 0.0010, 0.0003, 0.0082,
        0.0007, 0.0000, 0.0117, 0.0010, 0.0000, 0.0065, 0.0014, 0.0034, 0.0051,
        0.0000, 0.0000, 0.0010, 0.0000, 0.0072, 0.0000, 0.0000, 0.0124, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0178,
        0.0127, 0.0137, 0.0017, 0.0113, 0.0003, 0.0000, 0.0237, 0.0000, 0.0000,
        0.0000, 0.0312, 0.0151, 0.0168, 0.0000, 0.0000, 0.0000, 0.0072, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0062, 0.0271, 0.0124, 0.0000, 0.0003, 0.0086, 0.0003, 0.0000,
        0.0045, 0.0003, 0.0000, 0.0014, 

In [79]:
P = (N+1).float()
P/= P.sum(1, keepdims=True)
P[0].sum()

tensor(1.)

In [136]:
g = torch.Generator().manual_seed(1337)
ix = 0
for i in range(10):
    out = []
    while True:
        p = N[ix].float()
        p = p/ p.sum()
        ix = torch.multinomial(p, num_samples = 1, replacement = True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

ஷன்.
யோகலைநாதமிலிங்கோபித்.
அகில்ரா.
தி.
கோத்.
மோனி.
வரவர்.
துரை.
சோலைநாகவி.
கர்.


In [117]:
len(names_data)

2910

In [118]:
log_likelihood, c = 0.0, 1
for w in names_data:
    chs = ['.'] + w + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        p = P[ix1, ix2]
        logprob = torch.log(p)
        log_likelihood += logprob
        c += 1 
print(f'{log_likelihood=}')
print(f'{-log_likelihood/c}') # negative log likelihood 

log_likelihood=tensor(-55628.0586)
3.297454595565796


## Neural net

In [137]:
xs, ys = [], []
for w in names_data[:1]:
    chs = ['.'] + w + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        xs.append(ix1); ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [98]:
xs

tensor([  0,   2,  12,  81,  73, 128,  99])

In [99]:
ys

tensor([  2,  12,  81,  73, 128,  99,   0])

In [138]:
import torch.nn.functional as F

In [152]:
W = torch.rand((209, 209), generator = g, requires_grad = True)
xenc = F.one_hot(xs, num_classes = 209).float()
logits = xenc @ W
counts = logits.exp()
probs = counts/ counts.sum(1, keepdim = True)
loss = -probs[np.arange(len(xs)), ys].log().mean()
loss

tensor(5.3713, grad_fn=<NegBackward0>)

In [143]:
# Train set
xs, ys = [], []
for w in names_data:
    chs = ['.'] + w + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        xs.append(ix1); ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [150]:
n = len(xs)
n

16869

In [154]:
W = None
W = torch.rand((209, 209), generator = g, requires_grad = True)

In [183]:
## Training
for i in range(50):
    #forward pass
    xenc = F.one_hot(xs, num_classes = 209).float()
    logits = xenc @ W
    counts = logits.exp()
    prob = counts/ counts.sum(1, keepdim=True)
    loss = -prob[np.arange(n), ys].log().mean()
    print(loss.item())

    ## backward pass
    W.grad = None
    loss.backward()

    ## update weights
    W.data += - 10 * W.grad

3.0152180194854736
3.014967441558838
3.0147173404693604
3.014467477798462
3.0142180919647217
3.0139689445495605
3.0137202739715576
3.0134716033935547
3.01322340965271
3.0129761695861816
3.012728691101074
3.012481689453125
3.012235403060913
3.011988878250122
3.0117428302764893
3.0114974975585938
3.0112524032592773
3.01100754737854
3.010762929916382
3.010518789291382
3.01027512550354
3.0100317001342773
3.0097885131835938
3.0095460414886475
3.009303331375122
3.009061574935913
3.008819818496704
3.008578300476074
3.0083372592926025
3.0080971717834473
3.0078563690185547
3.0076162815093994
3.0073766708374023
3.0071372985839844
3.0068984031677246
3.006659507751465
3.0064218044281006
3.006183385848999
3.005945920944214
3.0057084560394287
3.0054714679718018
3.005234718322754
3.0049986839294434
3.004762649536133
3.0045268535614014
3.0042917728424072
3.004056453704834
3.003821611404419
3.003587484359741
3.0033535957336426


In [188]:
#g = torch.Generator().manual_seed(1337)
for i in range(10):
    ix = 0
    out = []
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes = 209).float()
        
        # get the data from model
        with torch.no_grad():
            logits = xenc @ W
            counts = logits.exp()
            probs = counts / counts.sum(1, keepdim=True)
        ix = torch.multinomial(probs, num_samples = 1, replacement = True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))
        

மோவுப்மார்.
சத்துரேயெமைணு.
நாரேஷ்.
ஜோதிசோமௌகீலைரியெவ்.
குமாரி.
வி.
கோகிரி.
ஹனி.
ஹரி.
ராம்.


## dummy testing

In [70]:
set(tamil.utf8.get_letters(names_data[154]))

{'இ', 'கொ', 'க்', 'சை', 'டி'}

In [58]:
c_to_i = {c:i for i,c in enumerate(tamil.utf8.tamil_letters + [' ', ''])}

In [59]:
c_to_i

{'அ': 0,
 'ஆ': 1,
 'இ': 2,
 'ஈ': 3,
 'உ': 4,
 'ஊ': 5,
 'எ': 6,
 'ஏ': 7,
 'ஐ': 8,
 'ஒ': 9,
 'ஓ': 10,
 'ஔ': 11,
 'ஃ': 12,
 'க்': 13,
 'ச்': 14,
 'ட்': 15,
 'த்': 16,
 'ப்': 17,
 'ற்': 18,
 'ஞ்': 19,
 'ங்': 20,
 'ண்': 21,
 'ந்': 22,
 'ம்': 23,
 'ன்': 24,
 'ய்': 25,
 'ர்': 26,
 'ல்': 27,
 'வ்': 28,
 'ழ்': 29,
 'ள்': 30,
 'ஜ்': 31,
 'ஷ்': 32,
 'ஸ்': 33,
 'ஹ்': 34,
 'க': 35,
 'கா': 36,
 'கி': 37,
 'கீ': 38,
 'கு': 39,
 'கூ': 40,
 'கெ': 41,
 'கே': 42,
 'கை': 43,
 'கொ': 44,
 'கோ': 45,
 'கௌ': 46,
 'ச': 47,
 'சா': 48,
 'சி': 49,
 'சீ': 50,
 'சு': 51,
 'சூ': 52,
 'செ': 53,
 'சே': 54,
 'சை': 55,
 'சொ': 56,
 'சோ': 57,
 'சௌ': 58,
 'ட': 59,
 'டா': 60,
 'டி': 61,
 'டீ': 62,
 'டு': 63,
 'டூ': 64,
 'டெ': 65,
 'டே': 66,
 'டை': 67,
 'டொ': 68,
 'டோ': 69,
 'டௌ': 70,
 'த': 71,
 'தா': 72,
 'தி': 73,
 'தீ': 74,
 'து': 75,
 'தூ': 76,
 'தெ': 77,
 'தே': 78,
 'தை': 79,
 'தொ': 80,
 'தோ': 81,
 'தௌ': 82,
 'ப': 83,
 'பா': 84,
 'பி': 85,
 'பீ': 86,
 'பு': 87,
 'பூ': 88,
 'பெ': 89,
 'பே': 90,
 'பை': 91,
 'பொ': 92,
 'போ'

In [5]:
[list(c) for c in tamil.utf8.tamil_letters]

[['அ'],
 ['ஆ'],
 ['இ'],
 ['ஈ'],
 ['உ'],
 ['ஊ'],
 ['எ'],
 ['ஏ'],
 ['ஐ'],
 ['ஒ'],
 ['ஓ'],
 ['ஔ'],
 ['ஃ'],
 ['க', '்'],
 ['ச', '்'],
 ['ட', '்'],
 ['த', '்'],
 ['ப', '்'],
 ['ற', '்'],
 ['ஞ', '்'],
 ['ங', '்'],
 ['ண', '்'],
 ['ந', '்'],
 ['ம', '்'],
 ['ன', '்'],
 ['ய', '்'],
 ['ர', '்'],
 ['ல', '்'],
 ['வ', '்'],
 ['ழ', '்'],
 ['ள', '்'],
 ['ஜ', '்'],
 ['ஷ', '்'],
 ['ஸ', '்'],
 ['ஹ', '்'],
 ['க'],
 ['க', 'ா'],
 ['க', 'ி'],
 ['க', 'ீ'],
 ['க', 'ு'],
 ['க', 'ூ'],
 ['க', 'ெ'],
 ['க', 'ே'],
 ['க', 'ை'],
 ['க', 'ொ'],
 ['க', 'ோ'],
 ['க', 'ௌ'],
 ['ச'],
 ['ச', 'ா'],
 ['ச', 'ி'],
 ['ச', 'ீ'],
 ['ச', 'ு'],
 ['ச', 'ூ'],
 ['ச', 'ெ'],
 ['ச', 'ே'],
 ['ச', 'ை'],
 ['ச', 'ொ'],
 ['ச', 'ோ'],
 ['ச', 'ௌ'],
 ['ட'],
 ['ட', 'ா'],
 ['ட', 'ி'],
 ['ட', 'ீ'],
 ['ட', 'ு'],
 ['ட', 'ூ'],
 ['ட', 'ெ'],
 ['ட', 'ே'],
 ['ட', 'ை'],
 ['ட', 'ொ'],
 ['ட', 'ோ'],
 ['ட', 'ௌ'],
 ['த'],
 ['த', 'ா'],
 ['த', 'ி'],
 ['த', 'ீ'],
 ['த', 'ு'],
 ['த', 'ூ'],
 ['த', 'ெ'],
 ['த', 'ே'],
 ['த', 'ை'],
 ['த', 'ொ'],
 ['த', 'ோ'],
 ['த', 'ௌ'],
 ['ப']

In [6]:
print(list('கொ'))
list('கொ')

['க', 'ொ']


['க', 'ெ', 'ா']

In [7]:
with open('./tamil_names.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
    for line in lines:

        print(line)
        [c_to_i[c] for c in list(tamil.utf8.get_letters(line.strip()))]

அகத்தியன்

அகநேயன்

அகரன்

அகல்யா

அகிலன்

அகிலா

அகில்

அக்சை குமார்

அக்னிராஜ்

அக்னிஹோத்ரா

அங்கயற்கண்ணி

அங்கயற்கரசி

அசோக்

அஜயன்

அஜய்

அஜித்

அஞ்சலி

அஞ்சு

அஞ்சுஸ்ரீ

அண்ணாமலை

அதிசயன்

அதியன்

அதியமான்

அதீஸ்

அத்வைத்

அனந்த கிருஷ்ணன்

அனந்தன்

அனலரசு

அனாமிகன்

அனாமிகா

அனிதா

அனிருத்

அனிஷ்

அனுகிரிதி

அனுசகன்

அனுஜ்

அனுன்யன்

அனுராக்

அனுராஜ்

அனுஷன்

அனுஷா

அனுஷ்

அனுஸ்ரீ

அனேகன்

அன்னக்கிளி

அன்னம்

அன்பரசன்

அன்பரசி

அன்பானந்தன்

அன்பானந்தம்

அன்பு

அன்பு மணி

அன்புக்கனி

அன்புக்கொடி

அன்புச்செழியன்

அன்ஷிகா

அபாஸ்

அபிஜித்

அபிநயன்

அபிநயா

அபிநாத்

அபினவ்

அபினாஷ்

அபினாஸ்

அபினேயன்

அபிமன்யு

அபிராஜ்

அபிராமி

அபிலேஷ்

அபிலேஸ்

அபிஷேக்

அபூர்வம்

அமலா

அமல்

அமல்நாத்

அமல்ராஜ்

அமிதாப்

அமிர்தா

அமுதன்

அமுதரசி

அமுதவல்லி

அமுதவாணன்

அமுதா

அமுதாம்பிகா

அம்பலவாணன்

அம்பிகா

அம்மு

அம்மையப்பன்

அரசு

அரவிந்

அரிமா

அருட்ச்செல்வன்

அருணகிரி

அருணா

அருணாச்சலம்

அருணோதயன்

அருண்

அருண்குமார்

அருண்மொழி வர்மன்

அருண்ராஜ்

அருண்விஜய்

அருந்ததி

அருளரசன்

அருளரசி

அருள்

அ

KeyError: 'கொ'

In [8]:
with codecs.open('./tamil_names.txt', "r", "utf-8") as handle:
    lines = handle.readlines()

    for i, line in enumerate(lines):
        word = ''
        for letter in filter(utf8.istamil,utf8.get_letters_iterable(line)):
            word += letter
        print(i, word)

0 அகத்தியன்
1 அகநேயன்
2 அகரன்
3 அகல்யா
4 அகிலன்
5 அகிலா
6 அகில்
7 அக்சைகுமார்
8 அக்னிராஜ்
9 அக்னிஹோத்ரா
10 அங்கயற்கண்ணி
11 அங்கயற்கரசி
12 அசோக்
13 அஜயன்
14 அஜய்
15 அஜித்
16 அஞ்சலி
17 அஞ்சு
18 அஞ்சுஸ்ரீ
19 அண்ணாமலை
20 அதிசயன்
21 அதியன்
22 அதியமான்
23 அதீஸ்
24 அத்வைத்
25 அனந்தகிருஷ்ணன்
26 அனந்தன்
27 அனலரசு
28 அனாமிகன்
29 அனாமிகா
30 அனிதா
31 அனிருத்
32 அனிஷ்
33 அனுகிரிதி
34 அனுசகன்
35 அனுஜ்
36 அனுன்யன்
37 அனுராக்
38 அனுராஜ்
39 அனுஷன்
40 அனுஷா
41 அனுஷ்
42 அனுஸ்ரீ
43 அனேகன்
44 அன்னக்கிளி
45 அன்னம்
46 அன்பரசன்
47 அன்பரசி
48 அன்பானந்தன்
49 அன்பானந்தம்
50 அன்பு
51 அன்புமணி
52 அன்புக்கனி
53 அன்புக்கொடி
54 அன்புச்செழியன்
55 அன்ஷிகா
56 அபாஸ்
57 அபிஜித்
58 அபிநயன்
59 அபிநயா
60 அபிநாத்
61 அபினவ்
62 அபினாஷ்
63 அபினாஸ்
64 அபினேயன்
65 அபிமன்யு
66 அபிராஜ்
67 அபிராமி
68 அபிலேஷ்
69 அபிலேஸ்
70 அபிஷேக்
71 அபூர்வம்
72 அமலா
73 அமல்
74 அமல்நாத்
75 அமல்ராஜ்
76 அமிதாப்
77 அமிர்தா
78 அமுதன்
79 அமுதரசி
80 அமுதவல்லி
81 அமுதவாணன்
82 அமுதா
83 அமுதாம்பிகா
84 அம்பலவாணன்
85 அம்பிகா
86 அம்மு
87 அம்மையப்பன்
88 அரசு
89 அர